In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [2]:
"""
Not the way to go must implement the Dataset class or no

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""
# Load the data into pandas dataframes, encode and split dynamic/static
# trainframe = pd.read_csv('train.csv')
# testframe = pd.read_csv('test.csv')

activity_encode = {"WALKING":1,
       "WALKING_DOWNSTAIRS":2,
       "WALKING_UPSTAIRS":3,
       "LAYING":4,
       "SITTING":5,
       "STANDING":6
       }
# trainframe['Activity'] = trainframe['Activity'].map(activity_encode)
# testframe['Activity'] = testframe['Activity'].map(activity_encode)
# dynamic = {}
# static = {}
#
# dynamic["trainframe"], static["trainframe"] = [x for _, x in trainframe.groupby(trainframe['Activity'] > 3)]
# dynamic["testframe"], static["testframe"] = [x for _, x in testframe.groupby(testframe['Activity'] > 3)]

In [3]:
# get the validation set from the training set
#
# dynamic["trainset"], dynamic["validset"] = np.split(trainframe, [int(.2*len(trainframe)), int(.8*len(trainframe))])
# static["trainset"], static["validset"] = np.split(trainframe, [int(.2*len(trainframe)), int(.8*len(trainframe))])

# maybe dataframe to csv?
# implement dataset for this one
# or this
# https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/
# https://www.tutorialspoint.com/pytorch/pytorch_convolutional_neural_network.htm

In [4]:
# Data mm

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train['Activity'] = train['Activity'].map(activity_encode)
test['Activity'] = test['Activity'].map(activity_encode)

In [7]:
train_x = np.array(train)
train_y = train['Activity'].values
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)

In [8]:
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((5881, 563), (5881,)), ((1471, 563), (1471,)))

In [11]:
# converting training images into torch format
train_x = train_x.reshape((5881, 563))
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

(torch.Size([5881, 563]), torch.Size([5881]))

In [12]:
# converting validation images into torch format
val_x = val_x.reshape((1471, 563))
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([1471, 563]), torch.Size([1471]))

In [ ]:
# Models
# Understand the paper
class DynamicCNN(Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv1d(1, 4, kernel_size=3, stride=1, padding=1),
            Conv1d(4, 4, kernel_size=3, stride=1, padding=1),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 3)
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x